In [80]:
import pandas as pd
import os

data_path = '/Users/nvoronov/Downloads/src'

# Загрузка данных
contact_preference = pd.read_csv(os.path.join(data_path, 'contact_preference.csv'), sep=';')
customer_preferenced_contact_type = pd.read_csv(os.path.join(data_path, 'customer_preferenced_contact_type.csv'), sep=';')
customer = pd.read_csv(os.path.join(data_path, 'customer.csv'), sep=';')
customers_segments = pd.read_csv(os.path.join(data_path, 'customers_segmetns.csv'), sep=';')
location = pd.read_csv(os.path.join(data_path, 'location.csv'), sep=';')
product_feedback = pd.read_csv(os.path.join(data_path, 'product_feedback.csv'), sep=';')
product = pd.read_csv(os.path.join(data_path, 'product.csv'), sep=';')
purchase = pd.read_csv(os.path.join(data_path, 'purchase.csv'), sep=';')
segment = pd.read_csv(os.path.join(data_path, 'segment.csv'), sep=';')
service_feedback = pd.read_csv(os.path.join(data_path, 'service_feedback.csv'), sep=';')

# Объединение таблиц
customer['customer_id'] = customer['id'] 
purchase['customer_id'] = purchase['user_id'] 
product['product_id'] = product['id']
product['purchase_id'] = product['id']
product_feedback['purchase_id'] = pd.to_numeric(product_feedback['purchase_id'], errors='coerce')
product_feedback['feedback_id'] = product_feedback['id']
purchase['purchase_id'] = purchase['id']
user_data = (
    customer.merge(customers_segments, on='customer_id', how='left', suffixes=('', '_segment'))
            .merge(purchase, on='customer_id', how='left', suffixes=('', '_purchase'))
            .merge(product, on='product_id', how='left', suffixes=('', '_product'))
            .merge(product_feedback, on='purchase_id', how='left', suffixes=('', '_feedback'))
)

user_data['quantity'] = pd.to_numeric(user_data['quantity'], errors='coerce')
user_data['price'] = pd.to_numeric(user_data['price'], errors='coerce')


# Обработка данных
user_data['purchase_amount'] = user_data['quantity'] * user_data['price']
user_data['product_name'] = user_data['product_name'].fillna('')

user_summary = user_data.groupby('customer_id').agg(
    total_purchases=('purchase_id', 'count'),
    total_spent=('purchase_amount', 'sum'),
    average_rating=('rating', 'mean'),
    highest_rated_product=('rating', 'max'),
    most_popular_product_name=('product_name', lambda x: x),
    total_reviews=('feedback_id', 'count')
).reset_index()

user_summary['most_popular_product_name'] = user_summary['most_popular_product_name'].apply(lambda x: x[0] if type(x) != str  else '')
user_summary

,customer_id,total_purchases,total_spent,average_rating,highest_rated_product,most_popular_product_name,total_reviews
0,1,11,563.84,8.000000,10.0,Journey,6
1,2,10,398.34,8.125000,10.0,The Wolf of Wall Street,8
2,3,10,247.85,8.750000,10.0,The Shape of Water,4
3,4,14,359.27,7.625000,10.0,Dune,8
4,5,9,164.91,9.000000,9.0,Celeste,3
5,6,7,478.39,6.750000,8.0,Goodfellas,4
6,7,7,161.89,7.500000,10.0,Mad Max: Fury Road,4
7,8,13,433.32,7.000000,10.0,Dead by Daylight,9
8,9,15,337.29,7.857143,9.0,Tekken 7,7
9,10,10,267.85,8.000000,10.0,Dungeons & Dragons,3


In [81]:
def calculate_spending_share(user_id):
    # Фильтруем данные по пользователю
    user_purchases = purchase[purchase['customer_id'] == user_id]

    # Вычисляем общие траты
    purchase_by_user = user_purchases.merge(product, how='left', on='product_id')
    purchase_by_user['amount'] = purchase_by_user.apply(lambda x:  float(x['price']) * float(x['quantity']), axis=1)
    total_spent = purchase_by_user['amount'].sum()

    # Вычисляем траты по категориям
    category_spending = purchase_by_user[purchase_by_user['category_name'].isin(['Movies', 'Games', 'Books'])].groupby('category_name')['amount'].sum().reset_index()

    # Добавляем долю от общих трат
    category_spending['share'] = category_spending['amount'] / total_spent

    
    return category_spending

# Пример вызова функции для конкретного пользователя
user_id_example = 1  # Замените на нужный ID пользователя
spending_share = calculate_spending_share(user_id_example)
print(spending_share)

  category_name  amount     share
0         Books   38.96  0.069098
1         Games  494.90  0.877731
2        Movies   29.98  0.053171
